In [2]:
from math import sqrt

# x, y, b
nodes = [
    [0, 0, 0],  # depot
    [4, 1, 1],
    [1, 2, 1],
    [0, 5, 1],
    [-3, 3, 1],
    [-2, 1, 1],
    [-5, 1, 1],
    [-5, -1, 1],
    [-1, -3, 1],
    [3, -2, 1],
    [6, -1, 1]
]
nLocations = len(nodes)
K = 3
Q = 4
T = 16

distance = {}
b = []
for i in range(nLocations):
    b.append(nodes[i][2])
    x0 = nodes[i][0]
    y0 = nodes[i][1]
    for j in range(nLocations):
        x1 = nodes[j][0]
        y1 = nodes[j][1]
        
        distance[i, j] = sqrt((x0 - x1) ** 2 + (y0 - y1) **2)
        print(str(round(distance[i, j], 2))+"\t", end=" ")
    print()

0.0	 4.12	 2.24	 5.0	 4.24	 2.24	 5.1	 5.1	 3.16	 3.61	 6.08	 
4.12	 0.0	 3.16	 5.66	 7.28	 6.0	 9.0	 9.22	 6.4	 3.16	 2.83	 
2.24	 3.16	 0.0	 3.16	 4.12	 3.16	 6.08	 6.71	 5.39	 4.47	 5.83	 
5.0	 5.66	 3.16	 0.0	 3.61	 4.47	 6.4	 7.81	 8.06	 7.62	 8.49	 
4.24	 7.28	 4.12	 3.61	 0.0	 2.24	 2.83	 4.47	 6.32	 7.81	 9.85	 
2.24	 6.0	 3.16	 4.47	 2.24	 0.0	 3.0	 3.61	 4.12	 5.83	 8.25	 
5.1	 9.0	 6.08	 6.4	 2.83	 3.0	 0.0	 2.0	 5.66	 8.54	 11.18	 
5.1	 9.22	 6.71	 7.81	 4.47	 3.61	 2.0	 0.0	 4.47	 8.06	 11.0	 
3.16	 6.4	 5.39	 8.06	 6.32	 4.12	 5.66	 4.47	 0.0	 4.12	 7.28	 
3.61	 3.16	 4.47	 7.62	 7.81	 5.83	 8.54	 8.06	 4.12	 0.0	 3.16	 
6.08	 2.83	 5.83	 8.49	 9.85	 8.25	 11.18	 11.0	 7.28	 3.16	 0.0	 


In [3]:
import gurobipy as gp
from gurobipy import GRB

model = gp.Model()

x = {}
for i in range(nLocations):
    for j in range(nLocations):
        for k in range(K):
            x[i, j, k] = model.addVar(vtype=GRB.BINARY)
y = {}
for i in range(nLocations):
    for k in range(K):
        y[i, k] = model.addVar(vtype=GRB.BINARY)
z = {}
for i in range(nLocations):
    z[i] = model.addVar(vtype=GRB.CONTINUOUS, lb = 0)

# each location is reached
for j in range(1, nLocations):
    for k in range(K):
        model.addConstr(gp.quicksum(x[i, j, k] for i in range(nLocations)) == y[j, k])
# each location is left
for i in range(1, nLocations):
    for k in range(K):
        model.addConstr(gp.quicksum(x[i, j, k] for j in range(nLocations)) == y[i, k])
for i in range(nLocations):
    model.addConstr(gp.quicksum(x[i, i, k] for k in range(K)) == 0)
# sub-tour elimination
for i in range(1, nLocations):
    for j in range(1, nLocations):
        if not (i == j):
            model.addConstr(z[i] - z[j] + nLocations * gp.quicksum(x[i, j, k] for k in range(K)) <= nLocations - 1)
# cluster
for i in range(1, nLocations):
    model.addConstr(gp.quicksum(y[i, k] for k in range(K)) == 1)
model.addConstr(gp.quicksum(y[0, k] for k in range(K)) == K)
# capacities
for k in range(K):
    model.addConstr(gp.quicksum(b[i] * y[i, k] for i in range(1, nLocations)) <= Q)
    model.addConstr(gp.quicksum(distance[i, j] * x[i, j, k] for i in range(nLocations) for j in range(nLocations)) <= T)
            
model.setObjective(
    gp.quicksum(x[i, j, k]*distance[i, j] for i in range(nLocations) for j in range(nLocations) for k in range(K)), 
    GRB.MINIMIZE
)
            
model.optimize()

Set parameter Username
Academic license - for non-commercial use only - expires 2024-03-14
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: Intel(R) Core(TM) i7-10510U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 178 rows, 407 columns and 1596 nonzeros
Model fingerprint: 0xe9021bc1
Variable types: 11 continuous, 396 integer (396 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [2e+00, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Presolve removed 22 rows and 47 columns
Presolve time: 0.01s
Presolved: 156 rows, 360 columns, 1500 nonzeros
Variable types: 10 continuous, 350 integer (350 binary)

Root relaxation: objective 2.925473e+01, 144 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    Bes

In [4]:
pathDistance = 0
for k in range(K):
    runIndex = 0
    path = [runIndex]
    finished = False
    while(not finished):
        for i in range(nLocations):
            if x[runIndex, i, k].x == 1.0:
                path.append(i)
                pathDistance += distance[runIndex, i]
                runIndex = i
                finished = (i == 0)
                break
    print(f"path of vehicle {k}: {path}")
print(f"total distance: {pathDistance}")

path of vehicle 0: [0, 9, 10, 1, 0]
path of vehicle 1: [0, 8, 7, 6, 5, 0]
path of vehicle 2: [0, 4, 3, 2, 0]
total distance: 41.836380878915406
